In [1]:
!pip install streamlit

!pip install transformers
!pip install torch
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=032597ac22ae9f2eebc0544822eb264e96a2a87bd74e91a53c49b17e6430c332
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [2]:
!pip install --ignore-installed blinker
!pip install streamlit --upgrade --no-cache-dir

In [3]:
!pip install streamlit

import streamlit as st
from transformers import pipeline
import torch
from langdetect import detect

# ========================================
# SETUP PAGE
# ========================================
st.set_page_config(
    page_title="AI Business Assistant",
    page_icon="🤖",
    layout="wide"
)

# ========================================
# LOAD MODELS (FREE HUGGING FACE)
# ========================================
@st.cache_resource  # Cache so models load only once
def load_models():
    """Load all AI models - 100% FREE from Hugging Face"""

    # 1. Question Answering (FREE)
    qa_model = pipeline(
        "question-answering",
        model="deepset/roberta-base-squad2",  # Free, good quality
        device=-1  # CPU (use 0 for GPU if available)
    )

    # 2. Sentiment Analysis (FREE)
    sentiment_model = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment-latest",
        device=-1
    )

    # 3. Text Generation (FREE)
    generator = pipeline(
        "text-generation",
        model="distilgpt2",  # Small, fast, FREE
        device=-1
    )

    return qa_model, sentiment_model, generator

# Load models
with st.spinner("🚀 Loading AI models... (first time takes 2 min)"):
    qa_model, sentiment_model, generator = load_models()

# ========================================
# BUSINESS INFO (CUSTOMIZE THIS)
# ========================================
BUSINESS_CONTEXT = """
Business Name: Peshawar Fashion Store
Location: University Road, Peshawar
Contact: 0300-1234567

Store Hours:
- Monday to Saturday: 10 AM - 8 PM
- Sunday: 12 PM - 6 PM

Products:
- Men's Kurtas: Rs 800 - Rs 2500
- Women's Suits: Rs 1500 - Rs 5000
- Kids Clothes: Rs 500 - Rs 1500

Delivery:
- Free delivery in Peshawar for orders above Rs 2000
- Standard delivery: Rs 150
- Delivery time: 1-2 days

Payment Methods:
- Cash on Delivery
- Easypaisa: 0300-1234567
- JazzCash: 0300-1234567

Return Policy:
- 7-day return for unused items
- Exchange available within 3 days
- Items must have tags attached
"""

# ========================================
# SIDEBAR (INFO)
# ========================================
st.sidebar.title("🏪 Business Info")
st.sidebar.info("""
**Peshawar Fashion Store**

📍 University Road, Peshawar
📞 0300-1234567
⏰ 10 AM - 8 PM (Mon-Sat)

**Demo Features:**
✅ Smart Q&A
✅ Sentiment Detection
✅ Auto-Responses
✅ Urdu Support
""")

st.sidebar.markdown("---")
st.sidebar.caption("Built with ❤️ using Hugging Face (100% FREE)")

# ========================================
# MAIN APP
# ========================================
st.title("🤖 AI Business Assistant Demo")
st.markdown("### Smart Customer Support for Pakistani Businesses")

# Create tabs for different features
tab1, tab2, tab3 = st.tabs(["💬 Chat", "📊 Analytics", "ℹ️ About"])

# ========================================
# TAB 1: CHAT INTERFACE
# ========================================
with tab1:
    st.markdown("#### Ask me anything about the business!")

    # Example questions
    st.markdown("**Try these questions:**")
    col1, col2, col3 = st.columns(3)

    example_questions = [
        "What are your prices?",
        "What are your store hours?",
        "Do you deliver to my area?",
        "What is your return policy?",
        "What payment methods do you accept?",
        "Where is your store located?"
    ]

    # Display example questions as buttons
    selected_question = None
    for i, q in enumerate(example_questions[:3]):
        if col1.button(q, key=f"q1_{i}"):
            selected_question = q

    for i, q in enumerate(example_questions[3:]):
        if col2.button(q, key=f"q2_{i}"):
            selected_question = q

    # User input
    user_question = st.text_input(
        "Or type your question here:",
        value=selected_question if selected_question else "",
        placeholder="e.g., Do you have delivery?"
    )

    if user_question:
        with st.spinner("🤔 AI is thinking..."):
            # Get answer using QA model
            result = qa_model(
                question=user_question,
                context=BUSINESS_CONTEXT
            )

            # Get sentiment
            sentiment = sentiment_model(user_question)[0]

            # Detect language
            try:
                lang = detect(user_question)
                lang_emoji = "🇬🇧" if lang == "en" else "🇵🇰" if lang == "ur" else "🌍"
            except:
                lang = "unknown"
                lang_emoji = "🌍"

            # Display results
            col1, col2 = st.columns([3, 1])

            with col1:
                st.success(f"**Answer:** {result['answer']}")
                st.caption(f"Confidence: {result['score']:.2%}")

            with col2:
                # Show sentiment
                sentiment_emoji = "😊" if sentiment['label'] == 'positive' else "😐" if sentiment['label'] == 'neutral' else "😟"
                st.metric(
                    "Customer Mood",
                    sentiment['label'].title(),
                    sentiment_emoji
                )

            # Show detection details
            with st.expander("🔍 AI Detection Details"):
                st.write(f"**Language Detected:** {lang_emoji} {lang.upper()}")
                st.write(f"**Sentiment:** {sentiment['label']} ({sentiment['score']:.2%})")
                st.write(f"**Answer Confidence:** {result['score']:.2%}")

# ========================================
# TAB 2: ANALYTICS DASHBOARD
# ========================================
with tab2:
    st.markdown("#### Customer Interaction Analytics")

    # Simulated stats (you can make these real later)
    col1, col2, col3, col4 = st.columns(4)

    col1.metric("Total Questions", "127", "+12")
    col2.metric("Avg Response Time", "0.8s", "-0.3s")
    col3.metric("Satisfaction Rate", "94%", "+2%")
    col4.metric("Questions Answered", "119/127", "93.7%")

    st.markdown("---")

    # Most asked questions
    st.markdown("##### 🔥 Most Asked Questions")
    most_asked = {
        "Store hours?": 45,
        "Delivery available?": 32,
        "What are prices?": 28,
        "Return policy?": 15,
        "Payment methods?": 7
    }

    for question, count in most_asked.items():
        st.write(f"**{question}** - Asked {count} times")
        st.progress(count / 50)

    st.info("💡 **Insight:** Most customers ask about delivery. Consider adding prominent delivery info on homepage!")

# ========================================
# TAB 3: ABOUT
# ========================================
with tab3:
    st.markdown("#### About This Demo")

    st.markdown("""
    This AI Business Assistant demonstrates how small Pakistani businesses can:

    ✅ **Automate Customer Support** - Answer common questions 24/7
    ✅ **Understand Customer Mood** - Detect if customer is frustrated/happy
    ✅ **Support Multiple Languages** - Works with English and Urdu
    ✅ **Provide Instant Responses** - Average response time under 1 second
    ✅ **Save Time & Money** - Reduce support workload by 70-80%

    ---

    #### 🛠️ Technology Stack (100% Free)

    - **Frontend:** Streamlit
    - **AI Models:** Hugging Face Transformers
    - **QA Model:** RoBERTa trained on SQuAD 2.0
    - **Sentiment:** Twitter-RoBERTa
    - **Language:** DistilGPT2
    - **Cost:** $0 (completely free!)

    ---

    #### 📈 Business Impact

    **Before AI Assistant:**
    - 4-5 hours daily answering questions
    - Missed customers due to slow response
    - Limited to business hours only

    **After AI Assistant:**
    - 80% questions answered automatically
    - 24/7 availability
    - 30-40 hours saved per week
    - Better customer satisfaction

    ---

    #### 💰 Pricing Model

    - **Small Business:** Rs 2,500/month
    - **Medium Business:** Rs 5,000/month
    - **Enterprise:** Custom pricing

    **ROI:** Saves 30-40 hours/week = Worth Rs 20,000-40,000 in labor

    ---

    #### 📞 Contact

    Built by: **[Your Name]**
    Course: BS Artificial Intelligence
    University: University of Peshawar

    📧 Email: your.email@example.com
    📱 WhatsApp: 03XX-XXXXXXX
    💼 LinkedIn: [Your Profile]

    *Interested in implementing this for your business? Let's talk!*
    """)

    # Call-to-action button
    if st.button("🚀 Schedule a Demo", type="primary"):
        st.balloons()
        st.success("Thank you! Please WhatsApp me at 03XX-XXXXXXX to schedule your personalized demo!")

# ========================================
# FOOTER
# ========================================
st.markdown("---")
col1, col2, col3 = st.columns([2, 1, 2])
with col2:
    st.caption("Made with ❤️ in Peshawar")

2025-11-10 19:01:20.471 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:20.473 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:20.546 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-10 19:01:20.547 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:20.548 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:20.549 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:20.551 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
2025-11-10 19:01:45.144 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:45.145 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:45.146 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:45.148 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:45.149 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:45.150 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:45.151 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 19:01:45.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor

In [8]:
%%writefile app.py
import streamlit as st
from transformers import pipeline
from langdetect import detect, LangDetectException
import warnings
warnings.filterwarnings('ignore')

st.set_page_config(page_title="AI Business Assistant", page_icon="🤖", layout="wide")

@st.cache_resource(show_spinner=False)
def load_models():
    qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2", device=-1)
    sentiment_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest", device=-1)
    generator = pipeline("text-generation", model="distilgpt2", device=-1)
    return qa_model, sentiment_model, generator

BUSINESS_CONTEXT = """
Business Name: Peshawar Fashion Store
Location: University Road, Peshawar
Contact: 0300-1234567
Store Hours:
- Monday to Saturday: 10 AM - 8 PM
- Sunday: 12 PM - 6 PM
Products:
- Men's Kurtas: Rs 800 - Rs 2500
- Women's Suits: Rs 1500 - Rs 5000
- Kids Clothes: Rs 500 - Rs 1500
Delivery:
- Free delivery in Peshawar for orders above Rs 2000
- Standard: Rs 150 (1-2 days)
Payment:
- COD, Easypaisa, JazzCash (0300-1234567)
Return Policy:
- 7-day return for unused
- Exchange within 3 days; must have tags
"""

with st.spinner("🚀 Loading AI models (first run takes time)..."):
    qa_model, sentiment_model, generator = load_models()
    if qa_model is None:
        st.stop()

st.sidebar.title("🏪 Business Info")
st.sidebar.info("""
**Peshawar Fashion Store**
📍 University Road, Peshawar
📞 0300-1234567
⏰ 10 AM - 8 PM (Mon-Sat)

**Demo:**
✅ Q&A
✅ Sentiment
✅ Urdu Support
""")

st.title("🤖 AI Business Assistant Demo")
st.markdown("### Smart Customer Support for Pakistani Businesses")
tab1, tab2, tab3 = st.tabs(["💬 Chat", "📊 Analytics", "ℹ️ About"])

with tab1:
    st.markdown("#### Ask anything about the business!")
    col1, col2, col3 = st.columns(3)
    ex_qs = [
        "What are your prices?","What are your store hours?","Do you deliver to my area?", "What is your return policy?","What payment methods do you accept?","Where is your store located?"
    ]
    selected_question = None
    for i,q in enumerate(ex_qs[:2]):
        if col1.button(q,key=f"q1_{i}"): selected_question = q
    for i,q in enumerate(ex_qs[2:4]):
        if col2.button(q,key=f"q2_{i}"): selected_question = q
    for i,q in enumerate(ex_qs[4:]):
        if col3.button(q,key=f"q3_{i}"): selected_question = q

    user_question = st.text_input("Or type your question here:", value=selected_question or "", placeholder="e.g., Do you have delivery?")
    if user_question:
        with st.spinner("🤔 AI is thinking..."):
            try:
                result = qa_model(question=user_question, context=BUSINESS_CONTEXT)
                sentiment = sentiment_model(user_question)
                try:
                    lang = detect(user_question)
                    lang_emoji = "🇬🇧" if lang == "en" else "🇵🇰" if lang == "ur" else "🌍"
                except LangDetectException:
                    lang = "unknown"
                    lang_emoji = "🌍"

                col1, col2 = st.columns([3, 1])
                with col1:
                    st.success(f"**Answer:** {result['answer']}")
                    st.caption(f"Confidence: {result['score']:.2%}")
                with col2:
                    senti_emoji = "😊" if sentiment['label']=='positive' else "😐" if sentiment['label']=='neutral' else "😟"
                    st.metric("Customer Mood", sentiment['label'].title(), senti_emoji)
                with st.expander("🔍 AI Detection Details"):
                    st.write(f"**Language Detected:** {lang_emoji} {lang.upper()}")
                    st.write(f"**Sentiment:** {sentiment['label']} ({sentiment['score']:.2%})")
                    st.write(f"**Answer Confidence:** {result['score']:.2%}")
            except Exception as e:
                st.error(f"Error: {e}")
                st.info("Try a different question.")

with tab2:
    st.markdown("#### Customer Interaction Analytics")
    c1, c2, c3, c4 = st.columns(4)
    c1.metric("Total Questions", "127", "+12")
    c2.metric("Avg Response Time", "0.8s", "-0.3s")
    c3.metric("Satisfaction Rate", "94%", "+2%")
    c4.metric("Questions Answered", "119/127", "93.7%")
    st.markdown("---")
    st.markdown("##### 🔥 Most Asked Questions")
    most_asked = {"Store hours?": 45, "Delivery available?": 32, "What are prices?": 28, "Return policy?": 15, "Payment methods?": 7}
    for q,count in most_asked.items():
        st.write(f"**{q}** - Asked {count} times")
        st.progress(count/50)
    st.info("💡 Most customers ask about delivery. Make it prominent!")

with tab3:
    st.markdown("#### About This Demo")
    st.markdown("""
    This assistant helps small shops:
    ✅ Automate replies
    ✅ Detect sentiment
    ✅ Support Urdu/English
    ✅ Instant responses
    ---
    **Built for BS AI - University of Peshawar.**
    """)

st.markdown("---")
col1, col2, col3 = st.columns([2, 1, 2])  # First unpack
with col2:  # Then use it
    st.caption("Made with ❤️ in Peshawar")



Overwriting app.py


In [9]:
!pip install streamlit pyngrok

In [10]:
import os
from pyngrok import ngrok
import subprocess
import time

NGROK_TOKEN = "354gMNMHDNugC1F2niKD6ArEMKi_5WJVdChF63YpHSxzLM7Sw" # 🔒 YOUR TOKEN
ngrok.kill()
os.system(f'ngrok config add-authtoken {NGROK_TOKEN}')

print("🚀 Starting Streamlit server...")
proc = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501'])
time.sleep(10)
print("🌐 Creating public URL via ngrok...")
public_url = ngrok.connect(8501)
print("Your public URL:", public_url)
print("\n⚠️ Keep this cell running! Stopping it closes the app.")


🚀 Starting Streamlit server...
🌐 Creating public URL via ngrok...
Your public URL: NgrokTunnel: "https://preprandial-louetta-cosmetically.ngrok-free.dev" -> "http://localhost:8501"

⚠️ Keep this cell running! Stopping it closes the app.
